# Modell der hämodynamischen Antwort

### module importieren

In [ ]:
import numpy as np
from nilearn import image, input_data, plotting
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from nistats import hemodynamic_models

sns.set_context('poster')

### Dateiname des funktionellen Bildes

In [ ]:
epi_file = '../data/sub-01_task-imagery_run-02_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

In [ ]:
epi_file

### Daten aus Seed extrahieren

Koordinaten der interessierenden Region

In [ ]:
my_seeds = [ (0,-50,30) ]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
display = plotting.plot_anat('../data/sub-01_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz',
                             cut_coords=my_seeds[-1],
                             axes=ax)
display.add_markers(my_seeds,marker_size=300)
plt.show()

Extraktion der Daten aus der Region

In [ ]:
my_standard = True
my_detrend = True
my_smoo = 8

In [ ]:
seed_masker = input_data.NiftiSpheresMasker(seeds=my_seeds,
                                            radius=5,
                                            standardize=my_standard,
                                            detrend=my_detrend,
                                            smoothing_fwhm=my_smoo).fit()

In [ ]:
seed_data = seed_masker.transform(epi_file)

seed_df = pd.DataFrame(seed_data)
seed_df.columns = ['signal']

In [ ]:
seed_df.tail()

Check: wenn das Detrending funktioniert hat, dann muss der Signalverlauf über alle Volumen hinweg flach sein

In [ ]:
linreg_results = stats.linregress(seed_df.index,seed_df['signal'])

In [ ]:
print('Schnittpunkt der y-Achse=%.5f\nSteigung der Geraden=%.5f'%(linreg_results.intercept, linreg_results.slope))

In [ ]:
plt.figure(figsize=(16,6))

# die Daten der Seed-Region
plt.plot(seed_df)

# die Ergebnisse der linearen Regression
plt.plot([linreg_results.intercept+linreg_results.slope*x for x in seed_df.index],
         label='Regressionslinie')

plt.ylabel('z-transformiertes Signal')
plt.xlabel('Zeit in Volumen')
plt.legend()
sns.despine(trim=True)
plt.show()

### Studiendesign laden

In [ ]:
design_df = pd.read_csv('../data/sub-01_task-imagery_run-02_events.tsv',sep='\t',index_col=[2,4])
design_df = design_df.sort_index()
design_df

## Modell der hämodynamischen Antwort

In [ ]:
my_tr = 3.0
my_onset = 0.0
my_length = 32.0

In [ ]:
hrf_model = hemodynamic_models.spm_hrf(tr=my_tr,
                                   oversampling=my_tr,
                                   time_length=my_length+my_onset,
                                   onset=my_onset)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(hrf_model,'-o',label='erwartete messbare Antwort')
plt.axvline(my_onset,color='r',label='neuronales Feuern')
plt.axhline(0,color='k')
sns.despine()
plt.legend(loc=(1,0.5))
plt.xlabel('Zeit in Sekunden')
plt.show()

### Interaktives Verschieben der HRF

In [ ]:
from ipywidgets import interact

In [ ]:
def f(my_onset):
    hrf_model = hemodynamic_models.spm_hrf(tr=my_tr,
                                       oversampling=my_tr,
                                       time_length=my_length+my_onset,
                                       onset=my_onset)
    plt.figure(figsize=(8,5))
    plt.plot(hrf_model,'-o',label='erwartete messbare Antwort')
    plt.axvline(my_onset,color='r',label='neuronales Feuern')
    plt.axhline(0,color='k')
    sns.despine()
    plt.legend(loc=(1,0.5))
    plt.xlabel('Zeit in Sekunden')
    plt.show()

In [ ]:
interact(f,my_onset=(0.0,30.0));

### Hypothetisches Design

Eine Minute passiert nichts:

In [ ]:
beispiel_design = np.zeros(60)
beispiel_design

Ein Event bei Sekunde 30 hinzufügen

In [ ]:
beispiel_design[30] = 1.0

In [ ]:
beispiel_design

Liste mit Events:

In [ ]:
event_onsets = np.where(beispiel_design==1)[-1]
event_onsets

### Abbildung mit Events und erwarteter Aktivierung

In [ ]:
plt.plot(np.convolve(beispiel_design,hrf_model))
sns.rugplot(event_onsets,color='r',height=0.1,linewidth=5,alpha=0.8)
plt.xlabel('Zeit in Sekunden')
sns.despine()
plt.show()

### Noch mehr Events hinzufügen

In [ ]:
beispiel_design[3] = 1.0
beispiel_design[33] = 1.0
beispiel_design[34] = 1.0
beispiel_design[45] = 1.0

In [ ]:
event_onsets = np.where(beispiel_design==1)[-1]
event_onsets

### Abbildung die zeigt was passiert, wenn sich HRFs überlappen

In [ ]:
plt.figure(figsize=(16,6))
for event in event_onsets:
    single_hrf = np.zeros(60)
    single_hrf[event] = 1.0
    plt.plot(np.convolve(single_hrf,hrf_model),color='g',alpha=0.5)
plt.plot([],color='g',alpha=0.5,label='einzelne HRFs')
plt.plot(np.convolve(beispiel_design,hrf_model),label='Summe der HRFs')
sns.rugplot(event_onsets,color='r',height=0.1,linewidth=5,alpha=0.8,label='Neuronale Aktivität')
plt.legend(loc='best')
plt.xlabel('Zeit in Sekunden')
sns.despine()
plt.show()

### Design der Studie modellieren

Anzahl der Volumen in der Zeitserie

In [ ]:
number_of_volumes = image.load_img(epi_file).shape[-1]
number_of_volumes

Zeit in Sekunden = Anzahl Volumen x TR

In [ ]:
beispiel_design = np.zeros(int(number_of_volumes*my_tr))
beispiel_design.shape[-1]

Onsets und Duration der Ruhebedingung

In [ ]:
ruhe_onsets = design_df.loc['RUHE',['onset','duration']]
ruhe_onsets

Alle Onsets einer Bedingung

In [ ]:
def make_onsets(df,number_of_volumes=number_of_volumes,my_tr=my_tr):
    design = np.zeros(int(number_of_volumes*my_tr))
    # für jede Zeile in der Tabelle
    for start in range(df.shape[0]):
        # Inhalte der Zeile
        block = df.iloc[start]
        # Onsets und Durations extrahieren
        block_onset = block['onset']
        block_duration = block['duration']
        # Ab Onset so viele 1en einfügen wie der Block lang ist
        for event in range(block_duration):
            design[block_onset+event] = 1.
    return design

In [ ]:
beispiel_design = make_onsets(ruhe_onsets)

In [ ]:
beispiel_design

Original An/Aus Design (Boxcar) oder mit HRF verbunden

In [ ]:
boxcar_design_in_tr = beispiel_design[::int(my_tr)]
boxcar_design_in_tr

In [ ]:
hrf_design_in_tr = np.convolve(beispiel_design,hrf_model)[::int(my_tr)]

### Abbildung mit Daten und Modellen

In [ ]:
plt.figure(figsize=(16,8))

# die Daten der Seed-Region
plt.plot(seed_df,color='grey',alpha=0.5)
# die Ergebnisse der linearen Regression
plt.plot([linreg_results.intercept+linreg_results.slope*x for x in seed_df.index],
         label='Regressionslinie',color='y')

# ein-aus-Design
plt.plot(boxcar_design_in_tr,color='r',label='neuronaler Response')
# HRF-Design
plt.plot(hrf_design_in_tr,'b',label='erwartete hämodynamische Antwort')

plt.legend(loc='best')

plt.ylabel('z-transformiertes Signal')
plt.xlabel('Zeit in Volumen')
sns.despine(trim=True)
plt.show()

### Korrelation Daten mit Design

In [ ]:
seed_df.corrwith(pd.Series(boxcar_design_in_tr))

In [ ]:
seed_df.corrwith(pd.Series(hrf_design_in_tr))

### Korrelation mit anderen Bedingungen

In [ ]:
beispiel_design = make_onsets(design_df.loc['MOTORIK',['onset','duration']])
boxcar_design_in_tr = beispiel_design[::int(my_tr)]
hrf_design_in_tr = np.convolve(beispiel_design,hrf_model)[::int(my_tr)]

In [ ]:
plt.figure(figsize=(16,8))

# die Daten der Seed-Region
plt.plot(seed_df,color='grey',alpha=0.5)
# die Ergebnisse der linearen Regression
plt.plot([linreg_results.intercept+linreg_results.slope*x for x in seed_df.index],
         label='Regressionslinie',color='y')

# ein-aus-Design
plt.plot(boxcar_design_in_tr,color='r',label='neuronaler Response')
# HRF-Design
plt.plot(hrf_design_in_tr,'b',label='erwartete hämodynamische Antwort')

plt.legend(loc='best')

plt.ylabel('z-transformiertes Signal')
plt.xlabel('Zeit in Volumen')
sns.despine(trim=True)
plt.show()

### Korrelation jdes Voxels des Gehirns mit dem Design

Whole-Brain Maske definieren

In [ ]:
brain_masker = input_data.NiftiMasker(mask_img='../data/sub-01_task-imagery_run-02_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz',
                                            standardize=my_standard,
                                            detrend=my_detrend,
                                            smoothing_fwhm=my_smoo).fit()

Daten auslesen

Daten in Tabelle packen

Design spezifizieren

Korrelationen rechnen

Korrelationen in Hirnbild umwandeln

Hirnbild visualisieren